In [ ]:
import sys
import openpyxl
import pandas as pd
import jaydebeapi

# 定义数据库连接
def db_conn():
    url = 'jdbc:oracle:thin:@8.142.180.78:1521:MAEASDB'
    driver = 'oracle.jdbc.driver.OracleDriver'
    jarFile = 'D:/Tools/JDBC/ojdbc8.jar'
    userTo = 'test'
    passwordTo = 'test'
    global connTo
    connTo = jaydebeapi.connect(jclassname=driver,
                                url=url,
                                driver_args=[userTo, passwordTo],
                                jars=jarFile)

def handle_excel():
    #global sf
    global sfnn
    #global dfn
    global dfnn
    #global dfh
    global regon_code
    regon_code = "3201"  #南京
    #3201,南京
    """
    读取标准项目信息,files 提供美奥提供excel文件路径和名称
                   ，newitem新项目的sheet名称，hisitem 老项目的sheet名称
    对新项目和老项目的一级分类、二级分类和收费项目名称分别进行编码，完成后保存留底
    regon_code 城市代码，每次上线需要查eas的t_bd_ctrlunit表获取fnumber字段
    dfn 新项目高值耗材匹配
    dfh 老项目高值耗材匹配
    dfb 收费项目分类编码，如果有新增的分类需要在excel中添加后在执行
    """
    files = "D:/Works/ProjectDoc/09 上海M/04 构建/老项目高值耗材匹配&EAS导入数据整理-(南京).xlsx"
    newitem = "新项目高值耗材匹配"
    hisitem = "老项目高值耗材匹配"

    #读取数据
    dfn = pd.DataFrame(
        pd.read_excel(files,
                      sheet_name=newitem,
                      header=0,
                      usecols=[0, 1, 2, 6]))
    dfh = pd.DataFrame(
        pd.read_excel(files,
                      sheet_name=hisitem,
                      header=0,
                      usecols=[0, 1, 10, 6]))
    dfb = pd.DataFrame(
        pd.read_excel('D:/C/工作簿1.xlsx',
                      sheet_name='Sheet6',
                      header=0,
                      usecols=None,
                      dtype={2: str}))

    dfn.insert(0, "regon_code", regon_code)

    dfh.insert(0, "regon_code", regon_code)
    dfh.columns.values[4] = "治疗项目名称"
    dfh.insert(3, "治疗项目名称", dfh.pop("治疗项目名称"))

    dfb['大类编码'] = regon_code + dfb['大类编码']
    dfb['小类编码'] = regon_code + dfb['小类编码']

    # 这里已经merge好了
    # 新项目
    dfn = pd.merge(dfn, dfb, how='inner').rename(columns={
        '大类编码': '项目大类编码',
        '小类编码': '项目小类编码'
    })
    dfn.insert(1, '项目大类编码', dfn.pop('项目大类编码'))
    dfn.insert(3, '项目小类编码', dfn.pop('项目小类编码'))

    # 老项目
    dfh = pd.merge(dfh, dfb, how='inner').rename(columns={
        '大类编码': '项目大类编码',
        '小类编码': '项目小类编码'
    })
    dfh.insert(1, '项目大类编码', dfh.pop('项目大类编码'))
    dfh.insert(3, '项目小类编码', dfh.pop('项目小类编码'))
    
    """
    map 用法
    map需要对映射表做去重处理
    dfn.insert(0, '项目大类编码', dfn['项目大类'].map(dfb.set_index('项目大类')['大类编码'].drop_duplicates()))
    dfn.insert(2, '项目小类编码', dfn['项目小类'].map(dfb.set_index('项目小类')['小类编码'].drop_duplicates()))
    """
    sf = dfn.drop_duplicates(subset=["项目大类", "治疗项目名称"]).sort_values(
        by=["项目大类", "项目小类", "治疗项目名称"], axis=0)
    sfh = dfh.drop_duplicates(subset=["项目大类", "治疗项目名称"]).sort_values(
        by=["项目大类", "项目小类", "治疗项目名称"], axis=0)

    #分类排序，把序号形成字符方法2 lambda表达式
    #sf['治疗项目编码'] =sf['治疗项目名称'].groupby(sf['项目大类']).rank().apply(lambda x:str(x).split(".")[0].zfill(4))
    #分类排序，把序号形成字符方法1 浮点转成整数
    sf.insert(
        5, '治疗项目编码', sf["项目大类编码"] + '-N' + sf['治疗项目名称'].groupby(
            sf['项目大类']).rank().astype(int).astype(str).str.zfill(4))
    sf.pop("EAS对应物料编码")
    sfh.insert(
        5, '治疗项目编码', sfh["项目大类编码"] + '-H' + sfh['治疗项目名称'].groupby(
            sfh['项目大类']).rank().astype(int).astype(str).str.zfill(4))
    sfh.pop("EAS对应物料编码")

    dfn.insert(
        5, '治疗项目编码',
        dfn['治疗项目名称'].map(sf.set_index('治疗项目名称')['治疗项目编码'].drop_duplicates()))
    dfh.insert(
        5, '治疗项目编码',
        dfh['治疗项目名称'].map(sfh.set_index('治疗项目名称')['治疗项目编码'].drop_duplicates()))

    sfnn = pd.concat([sf, sfh], axis=0).drop_duplicates(
        subset=["治疗项目名称"]).reset_index(drop=True).sort_values("治疗项目编码")
    dfnn = pd.concat(
        [dfn, dfh],
        axis=0).drop_duplicates(subset=["治疗项目名称", "EAS对应物料编码"]).reset_index(
            drop=True).sort_values("治疗项目编码")
    # 保存处理完成的数据
    path = f'D:/C/{regon_code}高值收费项目导入.xlsx'
    dfnsheet = f"{regon_code}高值收费项目对应物料整理表"
    sfsheet = f"{regon_code}高值收费项目导入表"
    with pd.ExcelWriter(path, mode='w') as writer:
        dfnn.to_excel(writer,
                      sheet_name=dfnsheet,
                      index=False,
                      columns=[
                          "regon_code", "项目大类编码", "项目大类", "项目小类编码", "项目小类",
                          "治疗项目编码", "治疗项目名称", "EAS对应物料编码"
                      ])
        sfnn.to_excel(writer,
                      sheet_name=sfsheet,
                      index=False,
                      columns=[
                          "regon_code", "项目大类编码", "项目大类", "项目小类编码", "项目小类",
                          "治疗项目编码", "治疗项目名称"
                      ])

    sfnn = sfnn.rename(
        columns={
            'regon_code': 'FREGIONCODE',
            '项目大类编码': 'FCATEGORYNUMBER',
            '项目大类': 'FCATEGORYNAME',
            '项目小类编码': 'FCATEGORYSUBCODE',
            '项目小类': 'FCATEGORYSUBNAME',
            '治疗项目编码': 'FPAYITEMNUMBER',
            '治疗项目名称': 'FPAYITEMNAME'
        }).sort_values(by=["FCATEGORYNUMBER", "FPAYITEMNUMBER"], axis=0)

    dfnn.drop(["regon_code", '项目大类编码', '项目大类', '项目小类编码', '项目小类'],
              axis=1,
              inplace=True)
    dfnn = dfnn.rename(
        columns={
            '治疗项目编码': 'FITEM_NUMBER',
            '治疗项目名称': 'FITEM_NAME',
            'EAS对应物料编码': 'FMATERIAL_CODE'
        }).sort_values(by=['FITEM_NUMBER'], axis=0)


def clear_table():
    cursTo = connTo.cursor()
    cursTo.execute("TRUNCATE TABLE CT_CUS_PAYITEMSCATE_INIT")
    cursTo.execute("TRUNCATE TABLE CT_MAPPING")
    cursTo.execute("DELETE FROM CT_CUS_PAYITEMSCATE WHERE CFREGIONID in (SELECT FID from T_ORG_CTRLUNIT WHERE FNUMBER =" "'%s')" % (regon_code))
    cursTo.execute("DELETE FROM CT_CUS_PAYITEMSCATESUB WHERE CFREGIONID in (SELECT FID from T_ORG_CTRLUNIT WHERE FNUMBER =" "'%s')" % (regon_code))
    cursTo.execute("DELETE FROM CT_CUS_PAYITEMS WHERE CFREGIONID in (SELECT FID from T_ORG_CTRLUNIT WHERE FNUMBER =" "'%s')" % (regon_code))
# 定义把handle_excel函数处理的结果插入到数据库中
def handle_insertdb(name):
    """
    把excel处理完成的数据存入数据表
    CT_CUS_PAYITEMSCATE_INIT 表形成收费项目的一级、二级分类和收费项目明细，对应sfnn的dataframe
    CT_MAPPING 表形成收费项目和物料对应关系，对应dfnn的dataframe
    尝试把dataframe转成tuples元组类型，实验executemany begin
    """
    # clear_table()
    columns = sfnn.columns if name == 'CT_CUS_PAYITEMSCATE_INIT' else dfnn.columns
    # 获取要出入数据的列名称
    tmp = ''
    fields = ''
    length_columns = len(columns)
    for i in range(length_columns):  # 获取要插入的列的数量及插入代码
        tmp = tmp + ':' + str(i + 1)
        fields = fields + columns[i]
        if i < length_columns - 1:
            tmp += ','
            fields += ','
            # 要执行插入操作的SQL
    insert_sql = f'''insert into {name} ({fields}) values({tmp}) '''
    print('datafram insert', insert_sql)

    insert_data = []
    if name == 'CT_CUS_PAYITEMSCATE_INIT':
        for rows in sfnn.itertuples():  # 遍历要插入的数据DataFrame
            row = rows[1:]  # 获取需要插入的数据。注意：rows[0]是索引。
            insert_data.append(row)
        try:
            print("在执行把收费项目excel导入数据库中...")
            cursTo = connTo.cursor()
            cursTo.executemany(insert_sql, insert_data)
            effectRow = cursTo.rowcount
            # insert_data = []
            print(name, f"收费项目整理表插入执行成功{str(effectRow)}" + "\n")
        except Exception as e:
            isSucceed = 0
            print('收费项目整理表插入执行过程有问题:', e)
        else:
            isSucceed = 1
            handle_categoryid()
        finally:
            cursTo.close()
    else:
        for rows in dfnn.itertuples():  # 遍历要插入的数据DataFrame
            row = rows[1:]  # 获取需要插入的数据。注意：rows[0]是索引。
            insert_data.append(row)
        try:
            print("在执行收费项目物料对应表导入中...")
            cursTo = connTo.cursor()
            cursTo.executemany(insert_sql, insert_data)
            effectRow = cursTo.rowcount
            # insert_data = []
            print(name, f"导入收费项目物料对应表成功{str(effectRow)}" + "，需要EAS进行处理导入\n")
        except Exception as e:
            isSucceed = 0
            print('收费项目整理表插入执行过程有问题:', e)
        finally:
            cursTo.close()


# #尝试把ddataframe转成tuples元组类型，实验executemany end
def handle_categoryid():
    cursTo = connTo.cursor()
    # 一级分类
    sqlStr = "INSERT INTO CT_CUS_PAYITEMSCATE(FID, FNUMBER, FNAME_L2, CFREGIONID, FCREATORID, FCREATETIME, FLASTUPDATEUSERID,FLASTUPDATETIME, FCONTROLUNITID) SELECT NEWBOSID('F699FE2C'),FCATEGORYNUMBER,FCATEGORYNAME,CTRL.FID,'256c221a-0106-1000-e000-10d7c0a813f413B7DE7F',sysdate,'256c221a-0106-1000-e000-10d7c0a813f413B7DE7F',sysdate,'00000000-0000-0000-0000-000000000000CCE7AED4'"
    sqlStr = sqlStr + " FROM (SELECT DISTINCT FCATEGORYNUMBER, FCATEGORYNAME, FREGIONCODE FROM CT_CUS_PAYITEMSCATE_INIT) S LEFT JOIN T_ORG_CTRLUNIT CTRL ON S.FREGIONCODE = CTRL.FNUMBER WHERE FREGIONCODE =" "'%s'" % (
        regon_code) + "  ORDER BY FCATEGORYNUMBER"
    try:
        print("在执行一级分类插入中...")
        cursTo.execute(sqlStr)
        effectRow = cursTo.rowcount
        isSucceed = 1
        print(regon_code, f"一级分类插入成功{str(effectRow)}" + "\n")
    except Exception as e:
        isSucceed = 0
        print('一级分类插入执行过程有问题:', e)


    sqlStr = ""
    sqlStr = "UPDATE CT_CUS_PAYITEMSCATE_INIT T SET FCATEGORYID=(SELECT FID FROM CT_CUS_PAYITEMSCATE S WHERE T.FCATEGORYNUMBER=S.FNUMBER) WHERE T.FCATEGORYID IS NULL"
    if isSucceed == 1:
        try:
            print("在更新辅助表的一级分类ID中...")
            cursTo.execute(sqlStr)
            effectRow = cursTo.rowcount
            isSucceed = 1
            print(regon_code, f"更新辅助表的一级分类ID成功{str(effectRow)}" + "\n")
        except Exception as e:
            isSucceed = 0
            print('更新辅助表的一级分类ID执行过程有问题:', e)
            sys.exit(1)
   
# 3.1 插入二级分类
    sqlStr = ""
    sqlStr = "INSERT INTO CT_CUS_PAYITEMSCATESUB(FID,FNUMBER, FNAME_L2, CFREGIONID, CFCATEGORYID, FCREATORID, FCREATETIME, FLASTUPDATEUSERID, FLASTUPDATETIME, FCONTROLUNITID)   SELECT NEWBOSID('4A528874'), FCATEGORYSUBCODE,FCATEGORYSUBNAME,CTRL.FID,FCATEGORYID,'256c221a-0106-1000-e000-10d7c0a813f413B7DE7F',sysdate,'256c221a-0106-1000-e000-10d7c0a813f413B7DE7F',sysdate,'00000000-0000-0000-0000-000000000000CCE7AED4'"
    sqlStr = sqlStr + " FROM (SELECT DISTINCT FCATEGORYNUMBER, FCATEGORYNAME, FREGIONCODE, FCATEGORYSUBCODE, FCATEGORYSUBNAME, FCATEGORYID FROM CT_CUS_PAYITEMSCATE_INIT) S LEFT JOIN T_ORG_CTRLUNIT CTRL ON S.FREGIONCODE = CTRL.FNUMBER where FREGIONCODE =" "'%s'" % (
        regon_code) + " ORDER BY FCATEGORYNUMBER"
    if isSucceed == 1:
        try:
            print("在执行二级分类插入中...")
            cursTo.execute(sqlStr)
            effectRow = cursTo.rowcount
            isSucceed = 1
            print(regon_code, f"二级分类插入成功{str(effectRow)}" + "\n")
        except Exception as e:
            isSucceed = 0
            print('二级分类插入执行过程有问题:', e)
            sys.exit(1)
   
# 3.2 更新辅助表的二级分类ID
    sqlStr = ""
    sqlStr = "UPDATE CT_CUS_PAYITEMSCATE_INIT T SET FCATEGORYSUBID=(SELECT FID FROM CT_CUS_PAYITEMSCATESUB S WHERE T.FCATEGORYSUBCODE=S.FNUMBER) WHERE T.FCATEGORYSUBID IS NULL"
    if isSucceed == 1:
        try:
            print("在执行更新辅助表的二级分类ID中...")
            cursTo.execute(sqlStr)
            effectRow = cursTo.rowcount
            isSucceed = 1
            print(regon_code, f"更新辅助表的二级分类ID成功{str(effectRow)}" + "\n")
        except Exception as e:
            isSucceed = 0
            print('更新辅助表的二级分类ID执行过程有问题:', e)
            sys.exit(1)
  
  # ************四、根据辅助表形成最终的收费项目*************/
# 4.1、收费项目插入
    sqlStr = ""
    sqlStr = "INSERT INTO CT_CUS_PAYITEMS(FID, FNUMBER, FCREATORID, FCREATETIME, FLASTUPDATEUSERID, FLASTUPDATETIME, FCONTROLUNITID, CFCATESUBID, FNAME_L2, CFUSESTATUS, CFISLOWVAL, CFREGIONID)  SELECT NEWBOSID('C6A0F53D'),FPAYITEMNUMBER,'256c221a-0106-1000-e000-10d7c0a813f413B7DE7F', sysdate, '256c221a-0106-1000-e000-10d7c0a813f413B7DE7F', sysdate,'00000000-0000-0000-0000-000000000000CCE7AED4',FCATEGORYSUBID,FPAYITEMNAME,'0',FPAYITETYPE,CTRL.FID"
    sqlStr = sqlStr + " FROM  CT_CUS_PAYITEMSCATE_INIT S LEFT JOIN T_ORG_CTRLUNIT CTRL ON S.FREGIONCODE = CTRL.FNUMBER WHERE FREGIONCODE=" "'%s'" % (
        regon_code) + " ORDER BY FCATEGORYNUMBER"
    if isSucceed == 1:
        try:
            print("在执行收费项目插入中...")
            cursTo.execute(sqlStr)
            effectRow = cursTo.rowcount
            print(regon_code, f"收费项目插入成功{str(effectRow)}" + "\n")
        except Exception as e:
            print('收费项目插入执行过程有问题:', e)
            sys.exit(1)
        finally:
            cursTo.close()
 
if __name__ == '__main__':
    handle_excel()
    db_conn()
    clear_table()
    handle_insertdb('CT_CUS_PAYITEMSCATE_INIT')
    handle_insertdb('CT_MAPPING')